In [3]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB 435.7 kB/s eta 0:01:30
   ---------------------------------------- 0.1/38.8 MB 409.6 kB/s eta 0:01:35
   ---------------------------------------- 0.1/38.8 MB 654.9 kB/s eta 0:01:00
   ---------------------------------------- 0.2/38.8 MB 748.1 kB/s eta 0:00:52
   ---------------------------------------- 0.2/38.8 MB 850.1 kB/s eta 0:00:46
   ---------------------------------------- 0.3/38.8 MB 842.9 kB/s eta 0:00:46
   ---------------------------------------- 0.4/38.8 MB 1.1 MB/s eta 0:00:35
    --------------------------------------- 0.5/38.8 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.8/38.8 MB 1.6 MB/s eta 0:00:24
   - -------------------------------------- 1.1/38.8 MB 2.1 MB/s eta 0:00:18
   - -------------------------------------- 1.4/38.8 MB 2.5 MB/s eta 0:00:15
 

In [1]:
!pip install tensorflow

  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl (7.5 kB)


In [104]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

img_size = 128
categories = ['transparent', 'colorful', 'black']
data = []

# Load and process images
for label, category in enumerate(categories):
    path = os.path.join(r"C:\Users\LINK TRADERS LT\Desktop\Data", category)
    for img_name in os.listdir(path):
        img = cv2.imread(os.path.join(path, img_name))
        img = cv2.resize(img, (img_size, img_size)) / 255.0
        data.append((img, label))

# Shuffle and split the data
np.random.shuffle(data)
X, y = zip(*data)
X, y = np.array(X), to_categorical(y, num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15)


In [105]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [108]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
'''
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)'''
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    
)




In [110]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=15,
                    validation_data=(X_val, y_val))


Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 359ms/step - accuracy: 0.3436 - loss: 1.2681 - val_accuracy: 0.3846 - val_loss: 1.0617
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 339ms/step - accuracy: 0.4937 - loss: 1.0210 - val_accuracy: 0.2821 - val_loss: 1.2086
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 378ms/step - accuracy: 0.4846 - loss: 0.8873 - val_accuracy: 0.5128 - val_loss: 0.7939
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 380ms/step - accuracy: 0.6296 - loss: 0.6581 - val_accuracy: 0.7692 - val_loss: 0.4555
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 334ms/step - accuracy: 0.8837 - loss: 0.4986 - val_accuracy: 0.9231 - val_loss: 0.5107
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 332ms/step - accuracy: 0.9240 - loss: 0.3604 - val_accuracy: 0.8974 - val_loss: 0.3314
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 362ms/step - accuracy: 0.9456 - loss: 0.2011 - val_accuracy: 0.9487 - val_loss: 0.3680
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 381ms/step - accuracy: 0.9354 - loss: 0.2044 - val_accuracy: 0.8974 - val_loss:

In [115]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9600 - loss: 0.3261 
Test Accuracy: 0.96


In [114]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame
    img = cv2.resize(frame, (img_size, img_size)) / 255.0
    img = np.expand_dims(img, axis=0)

    # Make predictions
    prediction = model.predict(img)
    class_index = np.argmax(prediction)
    class_label = categories[class_index]

    # Display prediction on frame
    cv2.putText(frame, class_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Object Classification", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━